# Challenge 1 - ret2win
hosted at the ROP Emporium



## ROP Emporium
</h4>This is a great place to learn about Return Oriented Programming, one of the sexier buffer overflow techniques used today. This is a CTF site where you attempt to exploit a binary using ROP in order to read out flag.txt </h4>

<a href="https://ropemporium.com/" target="_blank"><img src="https://1.bp.blogspot.com/-5Py6qw3W7mk/XxubymSzn6I/AAAAAAAAAok/5NjfpufdrJ0nHYcmvS8B9QHJMPw-DaS4ACLcBGAsYHQ/s1372/Screen%2BShot%2B2020-07-24%2Bat%2B10.40.44%2BPM.png" 
alt="FOOL, THE IMAGE IS GONE" width="240" height="180" border="10" /></a>

<h4>ROP Emporium offers their binaries in 4 architectural flavors, to start out I'll be using x86_64 (very basic I know) but I plan to run through the gauntlet again at least one more time to learn about ARM Pointer Auth Codes.</h4>



#
# Objective: Locate a method that you want to call within the binary. Call it by overwriting a saved return address on the stack.



# 
<h3>Starting off with a little file inspection:</h3>



In [38]:
import os
cmd = "file /home/Lab-Notes/ROP/'1 - ret2win'/ret2win"
os.system(cmd)

/home/Lab-Notes/ROP/1 - ret2win/ret2win: ELF 64-bit LSB executable, x86-64, version 1 (SYSV), dynamically linked, interpreter /lib64/ld-linux-x86-64.so.2, for GNU/Linux 3.2.0, BuildID[sha1]=19abc0b3bb228157af55b8e16af7316d54ab0597, not stripped


0

<h3>
So were dealing with a 64bit ELF (Executable and Linkable Format) in little endian (LSB stands for Least Significant Byte) we can also check this manually by looking at the sixth byte in the ELF header using a debugger.
</h3>
<h3>
The file is not stripped meaning the author of the exe compiled without stripping their function names, so the next step would be to check all the named functions in the binary using radare2 in hopes of finding some leads.
</h3>


# 
 



In [53]:
import os
cmd = "r2 -AAA ret2win -c afl"
os.system(cmd)

0x004005b0    1 42           entry0
0x004005f0    4 42   -> 37   sym.deregister_tm_clones
0x00400620    4 58   -> 55   sym.register_tm_clones
0x00400660    3 34   -> 29   entry.fini0
0x00400690    1 7            entry.init0
0x004006e8    1 110          sym.pwnme
0x00400580    1 6            sym.imp.memset
0x00400550    1 6            sym.imp.puts
0x00400570    1 6            sym.imp.printf
0x00400590    1 6            sym.imp.read
0x00400756    1 27           sym.ret2win
0x00400560    1 6            sym.imp.system
0x004007f0    1 2            sym.__libc_csu_fini
0x004007f4    1 9            sym._fini
0x00400780    4 101          sym.__libc_csu_init
0x004005e0    1 2            sym._dl_relocate_static_pie
0x00400697    1 81           main
0x004005a0    1 6            sym.imp.setvbuf
0x00400528    3 23           sym._init
 -- Jingle sploits, jingle sploits, ropchain all the way.
[0x004005b0]>   

[x] Analyze all flags starting with sym. and entry0 (aa)
[x] Analyze function calls (aac)
[x] Analyze len bytes of instructions for references (aar)
[x] Check for objc references
[x] Check for vtables
[x] Type matching analysis for all functions (aaft)
[x] Propagate noreturn information
[x] Use -AA or aaaa to perform additional experimental analysis.
[x] Finding function preludes
[x] Enable constraint types analysis for variables


0

<h3>
I'm not going to explain the command arguments, everything is pretty cut and dry (plus the man page exists). What it spits out is a memory address of our location (at entry0 in r2 which corresponds to the _start function in C based programs) within the binary and a list of functions for us to analyze.
</h3>

<h3>
We can see that there are indeed a few interesting functions: <strong>pwnme, ret2win,</strong> and <strong> main</strong></h3> 